In [38]:
import requests
import os

In [39]:
base_url = 'http://api.open-notify.org/iss-now.json'

In [40]:
# Make a get request to get the latest position of the international space station from the opennotify api.
response = requests.get(base_url)

# Print the status code of the response.
print(response.status_code)

200


#### Status codes indicate information about what happened with a request. Here are some codes that are relevant to GET requests:

#### 200 -- everything went okay, and the result has been returned (if any)
####  301 -- the server is redirecting you to a different endpoint. This can happen when a company switches domain names, or an endpoint name is changed.
#### 401 -- the server thinks you're not authenticated. This happens when you don't send the right credentials to access an API (we'll talk about authentication in a later post).
#### 400 -- the server thinks you made a bad request. This can happen when you don't send along the right data, among other things.
#### 403 -- the resource you're trying to access is forbidden -- you don't have the right permissions to see it.
#### 404 -- the resource you tried to access wasn't found on the server.

## We'll now make a GET request to http://api.open-notify.org/iss-pass, an endpoint that doesn't exist, per the API documentation.



In [41]:
response = requests.get("http://api.open-notify.org/iss-pass")
print(response.status_code)

404


## We'll now make a GET request to http://api.open-notify.org/iss-pass.json.

In [42]:
response = requests.get('http://api.open-notify.org/iss-pass.json')
print(response.status_code)

400


## Hitting the right endpoint:

#### We'll now make a GET request to http://api.open-notify.org/iss-pass.json.

In [43]:
response = requests.get("http://api.open-notify.org/iss-pass.json")
print(response.status_code)

400


## Query parameters: 

You'll see that in the last example, we got a 400 status code, which indicates a bad request. If you look at the documentation for the OpenNotify API, we see that the ISS Pass endpoint requires two parameters.We can do this by adding an optional keyword argument, "params", to our request. In this case, there are two parameters we need to pass:

- lat -- The latitude of the location we want.
- lon -- The longitude of the location we want.

In [60]:
parameters = {'lat':40.71,'lon':-74}

response = requests.get(base_url,params=parameters)

print(response.content)

# This will get the same response 

response = requests.get(base_url+'? lat=40.71&lon=-74')

print(response.content)

b'{"message": "success", "timestamp": 1549103835, "iss_position": {"longitude": "22.2504", "latitude": "3.2124"}}'
b'{"message": "success", "timestamp": 1549103835, "iss_position": {"longitude": "22.2685", "latitude": "3.2378"}}'


You may have noticed that the content of the response earlier was a string (although it was shown as a bytes object, we can easily convert the content to a string using response.content.decode("utf-8")).

In [62]:
response.content.decode('utf-8')

'{"message": "success", "timestamp": 1549103835, "iss_position": {"longitude": "22.2685", "latitude": "3.2378"}}'

In [72]:
response.json()['iss_position']

{'latitude': '3.2378', 'longitude': '22.2685'}

Strings are the way that we pass information back and forth to APIs, but it's hard to get the information we want out of them. How do we know how to decode the string that we get back and work with it in Python? How do we figure out the altitude of the ISS from the string response?

Luckily, there's a format called JavaScript Object Notation (JSON). JSON is a way to encode data structures like lists and dictionaries to strings that ensures that they are easily readable by machines. JSON is the primary format in which data is passed back and forth to APIs, and most API servers will send their responses in JSON format.



Python has great JSON support, with the json package. 
The json library has two main methods:

- dumps -- Takes in a Python object, and converts it to a string.
- loads -- Takes a JSON string, and converts it to a Python object.


In [84]:

# Make a list of fast food chains.
best_food_chains = ["Taco Bell", "Shake Shack", "Chipotle"]

# This is a list.
print(type(best_food_chains)) 

# Import the json library
import json

# Use json.dumps to convert best_food_chains to a string.
best_food_chains_string = json.dumps(best_food_chains)

# We've successfully converted our list to a string.
print(type(best_food_chains_string))

print(best_food_chains_string)

# Convert best_food_chains_string back into a list
print(type(json.loads(best_food_chains_string)))

# Make a dictionary
fast_food_franchise = {
    "Subway": 24722,
    "McDonalds": 14098,
    "Starbucks": 10821,
    "Pizza Hut": 7600
}

# We can also dump a dictionary to a string and load it.
fast_food_franchise_string = json.dumps(fast_food_franchise)

print(type(fast_food_franchise_string))

print(fast_food_franchise_string)

<class 'list'>
<class 'str'>
["Taco Bell", "Shake Shack", "Chipotle"]
<class 'list'>
<class 'str'>
{"Pizza Hut": 7600, "McDonalds": 14098, "Starbucks": 10821, "Subway": 24722}


## Getting JSON from an API request:

In [90]:
# Make the same request we did earlier, but with the coordinates of San Francisco instead.
parameters = {"lat": 37.78, "lon": -122.41}
response = requests.get("http://api.open-notify.org/iss-pass.json", params=parameters)

# Get the response data as a python object.  Verify that it's a dictionary.

data = response.json() # json converts the resposne data as python object

print(type(data),'\n')

print(data)

<class 'dict'> 

{'response': [{'risetime': 1549143064, 'duration': 605}, {'risetime': 1549148842, 'duration': 622}, {'risetime': 1549154741, 'duration': 510}, {'risetime': 1549160624, 'duration': 490}, {'risetime': 1549166421, 'duration': 603}], 'message': 'success', 'request': {'altitude': 100, 'passes': 5, 'datetime': 1549105389, 'latitude': 37.78, 'longitude': -122.41}}


## Content Type: 

The server doesn't just send a status code and the data when it generates a response. It also sends metadata containing information on how the data was generated and how to decode it. This is stored in the response headers. In Python, we can access this with the headers property of a response object.

In [101]:
# Headers is a dictionary
print(response.headers,'\n')

# Get the content-type from the dictionary.
print(response.headers["content-type"])

{'Content-Length': '521', 'Via': '1.1 vegur', 'Connection': 'keep-alive', 'Content-Type': 'application/json', 'Date': 'Sat, 02 Feb 2019 11:04:35 GMT', 'Server': 'nginx/1.10.3'} 

application/json


## Finding the number of people in space:

In [103]:
# Get the response from the API endpoint.

response = requests.get("http://api.open-notify.org/astros.json")

data = response.json()

# finding people are currently in space.
print(data["number"])

print(data)

3
{'people': [{'craft': 'ISS', 'name': 'Oleg Kononenko'}, {'craft': 'ISS', 'name': 'David Saint-Jacques'}, {'craft': 'ISS', 'name': 'Anne McClain'}], 'message': 'success', 'number': 3}
